In [1]:
import pandas as pd
import numpy as np
import re
from utils.tokenize_text import tokenize_text
from rouge_score import rouge_scorer
from nltk.tokenize import LineTokenizer, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
# Load dataset
df = pd.read_json("./data/wiki_geo.json").iloc[:1000]

In [3]:
# remove None
df.fillna(value=np.nan, inplace=True)
df.dropna(inplace=True)

In [4]:
# Print first line
df.head(1)

,categories,titles,summaries,contents,entities
0,géographie générale,Géographie,La géographie (du grec ancien γεωγραφία / geōg...,La géographie (du grec ancien γεωγραφία / geōg...,"[Alain Barré, Alphanumérique, American Geograp..."


In [5]:
# Articles number
df.shape

(436, 5)

In [6]:
END_OF_ARTICLES = ["== Références ==", "== Voir aussi ==", "== Liens externes =="]

In [7]:
# List chapter tags
def ls_chapter_tags(text, regex=r'^===* .* =*==$'):
  lines = text.split('\n')
  return [line for line in lines if re.match(regex, line)]
  

In [8]:
# Return true if tags is present in the article
def tags_are_present_in_article(text, tags):
  lines = text.split('\n')
  lines = [line.lower() for line in lines]
  for tag in tags:
    if tag.lower() in lines:
      return True
  return False

In [9]:
def index_of_tag_is_in_article(lines, tags):
  for i in range(len(lines)):
    for tag in tags:
      if tag.lower() == lines[i].lower():
        return i
  return None

# Return indices of tags
def trunc_article(lines, tags):
  idx = index_of_tag_is_in_article(lines, tags)
  if idx is not None:
    lines = lines[:idx]
  return lines

In [10]:
# get text flatted of article
def flat_article(content, summary, regex_tags=r'^===* .* =*==$'):
  # remove summary
  content = content[len(summary):]
  # split
  lines = content.split('\n')
  # trunc
  lines = trunc_article(lines, END_OF_ARTICLES)
  # remove tags
  lines = [line for line in lines if not re.match(regex_tags, line)]
  # merge string
  text = "\n".join(lines)
  return text

In [11]:
# return the content of an article without summary and footer
def content_article(content, summary):
  # remove summary
  content = content[len(summary):]
  # split
  lines = content.split('\n')
  # trunc
  lines = trunc_article(lines, END_OF_ARTICLES)
  # merge string
  text = "\n".join(lines)
  return text

In [12]:
# extract contents
articles_flat = [flat_article(df.iloc[i]["contents"], df.iloc[i]["summaries"]) for i in range(df.shape[0])]
trunc_contents = [content_article(df.iloc[i]["contents"], df.iloc[i]["summaries"]) for i in range(df.shape[0])]

In [13]:
# add to dataframe
df["flat_contents"] = articles_flat
df["trunc_contents"] = trunc_contents

In [14]:
# Tokenize flat contents
tokenized_flat_contents = [tokenize_text(df.iloc[i]["flat_contents"]) for i in range(df.shape[0])]

In [15]:
labels_entities = [[[0 for _ in line] for line in article] for article in tokenized_flat_contents]

In [16]:
# labelize token
for article in range(len(tokenized_flat_contents)):
  entities = df.iloc[article]["entities"]
  entities = sorted(entities, key=len, reverse=True)
  for entity in entities:
    entity_splited = entity.lower().split(' ')
    zeros = [0 for _ in entity_splited]
    for y, line in enumerate(tokenized_flat_contents[article]):
      for x, word in enumerate(line):
        if len(entity_splited) == len(line[x:x+len(entity_splited)]) and entity_splited == line[x:x+len(entity_splited)] and labels_entities[article][y][x:x+len(entity_splited)] == zeros:
          if len(entity_splited) >= 2:
            for i in range(len(entity_splited)):
              labels_entities[article][y][x+i] = 'C'
            labels_entities[article][y][x] = 'L'
            labels_entities[article][y][x+len(entity_splited) - 2] = 'R'
          else:
            labels_entities[article][y][x] = 'E'

  if article % 100 == 0:
    print("labels_entities: {}/{}".format(article+1, len(tokenized_flat_contents)))

labels_entities: 1/436
labels_entities: 101/436
labels_entities: 201/436
labels_entities: 301/436
labels_entities: 401/436


In [17]:
df["labels_entities"] = labels_entities

In [18]:
# labelize sentence
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def compute_labels_doc(doc, summary, is_sep_n = False):
  labels = []
  s = []
  # split doc
  if is_sep_n:
    nltk_line_tokenizer = LineTokenizer()
    s = nltk_line_tokenizer.tokenize(doc)
  else:
    s = sent_tokenize(doc, language="french")
  # compute labels
  if (len(s) > 0):
    a = ""
    score = scorer.score(a, summary)
    for i in range(len(s)):
      current_score = scorer.score(a + s[i] + ".", summary)
      if  current_score["rouge1"].fmeasure > score["rouge1"].fmeasure or \
        current_score["rouge2"].fmeasure > score["rouge2"].fmeasure or \
        current_score["rougeL"].fmeasure > score["rougeL"].fmeasure:
        score = current_score
        a = a + s[i] + "."
        labels.append(1)
      else:
        labels.append(0)

  return labels

In [20]:
labels_sentences = []

for i in range(df.shape[0]):
    labels_sentences.append(compute_labels_doc(df.iloc[i]["flat_contents"], df.iloc[i]["summaries"], is_sep_n=False))

    if i % 10 == 0:
        print("labels_sentences: {}/{}".format(len(labels_sentences), df.shape[0]))

labels_sentences: 1/436
labels_sentences: 11/436
labels_sentences: 21/436
labels_sentences: 31/436
labels_sentences: 41/436


In [ ]:
df["labels_sentences"] = labels_sentences

In [ ]:
# save to json
df.to_json("./data/wiki_geo_preprocessed.json")

In [ ]:
df.head(10)

,categories,titles,summaries,contents,entities,flat_contents,trunc_contents,labels_entities,labels_sentences
0,géographie générale,Géographie,La géographie (du grec ancien γεωγραφία – geog...,La géographie (du grec ancien γεωγραφία – geog...,"[Alain Barré, Alphanumérique, American Geograp...",\n\n\n\nLa première personne à utiliser le mot...,\n\n\n== Évolution et étendue de la notion de ...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, E, 0, 0], [...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, ..."
1,géographie générale,Épistémologie de la géographie,Cet article traite de l'épistémologie de la gé...,Cet article traite de l'épistémologie de la gé...,"[Aménagement du territoire, Analyse spatiale, ...",\n\n\nLa géographie classique française est ég...,\n\n\n== La Géographie vidalienne ou géographi...,"[[0, E, 0, 0, 0, 0, 0, E, 0, 0, 0, 0, L, C, C,...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,géographie générale,Coordonnées géographiques,Par coordonnées géographiques (ou encore « rep...,Par coordonnées géographiques (ou encore « rep...,"[100e méridien est, 100e méridien ouest, 101e ...","\n\n\n\nLa latitude est une valeur angulaire, ...",\n\n\n== Latitude ==\n\nLa latitude est une va...,"[[0, E, E, 0, 0, 0, 0, 0, 0, E, 0, E, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
3,géographie générale,Géographie physique,"La géographie physique, appelée aussi géograph...","La géographie physique, appelée aussi géograph...","[Afrique, Al Idrissi, Alexander von Humboldt, ...",\n\n\n\nLa géographie physique englobe plusieu...,\n\n\n== Disciplines ==\n\nLa géographie physi...,"[[0, E, E, 0, 0, 0, 0, E, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
4,géographie générale,Géographie vidalienne,Les principes de la géographie vidalienne,Les principes de la géographie vidalienne\n\n\...,"[1903, Albert Demangeon, André Cholley, Annale...",\n\n\nPaul Vidal de la Blache (22 janvier 1845...,\n\n\n== Paul Vidal de La Blache ==\nPaul Vida...,"[[L, C, C, R, C, 0, 0, 0, E, 0, 0, 0, E, 0, 0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,géographie générale,Position (géographie),"La position, en géographie est un point dans l...","La position, en géographie est un point dans l...","[Coordonnées cartésiennes, Coordonnées géograp...",\n\n\nUne position relative est une position p...,\n\n\n== Types de position ==\nUne position re...,"[[0, E, 0, 0, 0, E, 0, 0, 0, 0, 0, E], [0, 0, ...","[1, 1, 1, 0, 1]"
6,géographie générale,Marigot (géographie),Un marigot est une petite étendue d'eau fermée...,Un marigot est une petite étendue d'eau fermée...,"[Afrique subsaharienne, Amazonie, Bayou, Berna...","\n\n\nBernard Nantet, Dictionnaire de l'Afriqu...","\n\n\n== Bibliographie ==\nBernard Nantet, Dic...","[[R, C, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1]"
7,géographie générale,Géographie de la Bretagne,Cet article décrit la géographie de la Bretagn...,Cet article décrit la géographie de la Bretagn...,"[1842, 1968, 1969, 1970, 1er janvier, 2005, 20...",\n\n\n\n\nLa péninsule bretonne se situe à l’e...,\n\n\n== Géographie physique ==\n\n\n=== Situa...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, E, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, ..."
8,géographie générale,Géographie de Singapour,Singapour est une cité-État très urbanisée d'A...,Singapour est une cité-État très urbanisée d'A...,"[Altitude, Anglais, Années 1960, Années 2030, ...","\nPulau Ujong, l'ile principale, est en forme ...","\n== Géographie physique ==\nPulau Ujong, l'il...","[[R, C, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, ..."
9,géographie générale,Géographie de l'Équateur,L'Équateur est un pays situé en Amérique du Su...,L'Équateur est un pays situé en Amérique du Su...,"[Amazone (fleuve), Amérique, Amérique du Sud, ...",\n\n\n\nSa superficie est de 283 560 km2 dont ...,\n\n\n== Géographie générale du pays ==\n\nSa ...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[